spark-submit --master yarn --deploy-mode cluster  --conf spark.yarn.submit.waitAppCompletion=false --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.speculation=false --conf spark.executorEnv.LANG=en_US.UTF-8 --conf spark.yarn.appMasterEnv.LANG=en_US.UTF-8 --driver-cores 20 --driver-memory 50G --num-executors 20 --executor-cores 25 --executor-memory 30G pyspark-merge-tweets-similarity-scores.py

In [15]:
print('Running Time (in sec):')
(1579015886163-1579013225652)/(1000)

Running Time (in sec):


2660.511

In [1]:
import os
import sys
import socket
import re
import numpy as np
import string
from timeit import default_timer as timer
from datetime import datetime
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,dayofmonth,dayofweek,to_timestamp,size,isnan,lower,rand, lit
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType

In [2]:
print('Hostname:', socket.gethostname())
try:
    spark
except NameError:
    if 'samuel' in socket.gethostname().lower():
        print('Create Local SparkSession')
        spark = SparkSession.builder.config(
        "spark.driver.host", "localhost").appName(
        "merge-tweets-similarity-scores").getOrCreate()
    else:
        print('Create Cluster SparkSession')
        spark = SparkSession.builder.appName(
        "merge-tweets-similarity-scores").getOrCreate()
spark

Hostname: Samuels-MacBook-Pro.local


In [3]:
country_code = "US"
print('Country:', country_code)

# Local
if  'samuel' in socket.gethostname().lower():
    path_to_data = os.path.join('../../data/classification',country_code)
# Cluster
else:
    path_to_data = os.path.join('/user/spf248/twitter/data/classification',country_code)
print('Path to data:',path_to_data)

Country: US
Path to data: ../../data/classification/US


In [4]:
print('Import tweets containing keywords')
filtered = spark.read.parquet(os.path.join(path_to_data,'filtered'))
filtered.cache()

Import tweets containing keywords


DataFrame[tweet_id: string, text: string, fired: boolean, hired: boolean, job: boolean, laid_off: boolean, position: boolean, quit: boolean, unemployed: boolean, work: boolean, keyword: boolean]

In [5]:
print('Import random tweets')
random = spark.read.parquet(os.path.join(path_to_data,'random'))
random.cache()

Import random tweets


DataFrame[tweet_id: string, text: string, fired: boolean, hired: boolean, job: boolean, laid_off: boolean, position: boolean, quit: boolean, unemployed: boolean, work: boolean, keyword: boolean]

In [6]:
print('Import scores')

schema = StructType([StructField('tweet_id', StringType(), False),
                     StructField('score', FloatType(), False),
                     StructField('target', StringType(), False)])

scores = spark.read.option('header','true').schema(schema).csv(os.path.join(path_to_data,'similarity'))
scores.cache()

Import scores


DataFrame[tweet_id: string, score: float, target: string]

In [7]:
print('Drop Duplicated Scores (Random Sample Could Contain Keywords)')
scores = scores.drop_duplicates(subset=['tweet_id','target'])

Drop Duplicated Scores (Random Sample Could Contain Keywords)


# Merge Tweets With Their Similarity Scores

In [9]:
targets=sorted(scores.select("target").distinct().rdd.map(lambda r: r[0]).collect())
print('Targets:\n')
print('\n'.join(targets))

Targets:

I lost my job today


In [10]:
print('Merge Tweets With Their Similarity Scores:')
for target in targets:
    
    print(target)
    
    filtered=filtered.join(
    scores.filter(scores['target']==target).selectExpr(
    "tweet_id", 
    "score as target_"+target.replace(' ','_').replace('?','').lower()),on='tweet_id')
    
    random=random.join(
    scores.filter(scores['target']==target).selectExpr(
    "tweet_id", 
    "score as target_"+target.replace(' ','_').replace('?','').lower()),on='tweet_id')

Merge Tweets With Their Similarity Scores:
I lost my job today


In [11]:
print('Save')
filtered.write.mode("overwrite").parquet(os.path.join(path_to_data,'filtered-scored'))
random.write.mode("overwrite").parquet(os.path.join(path_to_data,'random-scored'))

Save
